# **Milestone 4 - LSTM and GloveVector word embeddings**

Ali Wehbe - Dina Younes - Shaza Fakih - Reeda El Saintbai - Youssef Jaafar

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Note:
 - make sure to upload the "embedding_matrix.pkl" and "iseardataset.csv" file in the same directory as this ipynb file.

**Downloading GloveVector**

Note: 
- make sure you create a folder named "embeddings" in the same directory of this .ipynb file before running the code below


In [25]:
import zipfile
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip      #needed for using the word embeddings GloveVector 
zip_file = zipfile.ZipFile('glove.840B.300d.zip')
zip_file.extractall('./embeddings/')

--2020-12-03 17:42:04--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2020-12-03 17:42:04--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2020-12-03 17:42:05--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

**Function used for cleaning words**




In [36]:
import re
def clean_words(str):                  #cleaning strings (removing excessive spaces, punctuations, etc)
    str = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", str)
    str = re.sub(r"\'s", " \'s", str)             #ex: "my friend's father" becomes "my friend 's father"
    str = re.sub(r"\'ve", " \'ve", str)           #ex: "I've" -> "I 've"
    str = re.sub(r"n\'t", " n\'t", str)           #ex: "don't" -> "do n't"
    str = re.sub(r"\'re", " \'re", str)           #ex: "they're" -> "they 're"
    str = re.sub(r"\'d", " \'d", str)             #ex: "I'd" becomes "I 'd"
    str = re.sub(r"\'ll", " \'ll", str)           #ex: "I'll" becomes "I 'll"
    str = re.sub(r",", " , ", str)        # here we split a word from punctuations: ex: "hello," becomes "hello ," and  "yes!" becomes "yes !" etc.
    str = re.sub(r"!", " ! ", str)         
    str = re.sub(r"\(", " \( ", str)
    str = re.sub(r"\)", " \) ", str)      #ex: "(for)" -> "\\( for \\)"
    str = re.sub(r"\?", " \? ", str)      #ex: "done?" -> "done \\?"
    str = re.sub(r"\s{2,}", " ", str)
    return str.strip().lower()           #converting to lower case  

#example
str = " I've broken my leg, while skiing the previous winter— "
print("example sentence1:")
print(str)
print("cleaned sentence1:")
print(clean_words(str))

str = "I suffered a lot!! Don't you want to help? (Please) "
print("example sentence2:")
print(str)
print("cleaned sentence2:")
clean_words(str)

example sentence1:
 I've broken my leg, while skiing the previous winter— 
cleaned sentence1:
i 've broken my leg , while skiing the previous winter
example sentence2:
I suffered a lot!! Don't you want to help? (Please) 
cleaned sentence2:


"i suffered a lot ! ! do n't you want to help \\? \\( please \\)"

In [26]:
import re
def clean_words(str):                  #cleaning strings (removing excessive spaces, punctuations, etc)
    str = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", str)
    str = re.sub(r"\'s", " \'s", str)             #ex: "my friend's father" becomes "my friend 's father"
    str = re.sub(r"\'ve", " \'ve", str)           #ex: "I've" becomes "I 've"
    str = re.sub(r"n\'t", " n\'t", str)           #ex: "don't" becomes "do n't"
    str = re.sub(r"\'re", " \'re", str)           #ex: "they're" -> "they 're"
    str = re.sub(r"\'d", " \'d", str)             #ex: "I'd" becomes "I 'd"
    str = re.sub(r"\'ll", " \'ll", str)           #ex: "I'll" becomes "I 'll"
    str = re.sub(r",", " , ", str)        # here we split a word from punctuations: ex: "hello," becomes "hello ," and  "yes!" becomes "yes !" etc.
    str = re.sub(r"!", " ! ", str)         
    str = re.sub(r"\(", " \( ", str)
    str = re.sub(r"\)", " \) ", str)      #ex: "(for)" -> "\\( for \\)"
    str = re.sub(r"\?", " \? ", str)      #ex: "done?" -> "done \\?"
    str = re.sub(r"\s{2,}", " ", str)
    return str.strip().lower()           #converting to lower case  

#example
str = " I've broken my leg skiing the previous winter— first time down the hill— I suffered a lot!! "
clean_words(str)


################################################################################
#Loading Data

import pandas as pd
from keras.utils.np_utils import to_categorical

DIR_DATA = '/content/drive/My Drive/Colab Notebooks/'
filename = 'iseardataset.csv'

df = pd.read_csv(DIR_DATA + filename)
needed = ['label', 'text']
not_needed = list(set(df.columns) - set(needed))
df = df.drop(not_needed, axis=1)
df = df.dropna(axis=0, how='any', subset=needed)
y_labels = sorted(list(set(df[needed[0]].tolist())))
dict.fromkeys(set(df[needed[0]].tolist()))
labels_dictionary = {}
for i in range(len(y_labels)):
  labels_dictionary[y_labels[i]] = i

x_train = df[needed[1]].apply(lambda x: clean_words(x)).tolist()      #cleaning sentences
y_train = df[needed[0]].apply(lambda y: labels_dictionary[y]).tolist()
y_train = to_categorical(np.asarray(y_train))

sentences = x_train
y_labels = y_train


################################################################################
# using GloveVector  word embeddings

import os
import pandas as pd
np.random.seed(7)
filename2 = 'glove.840B.300d.txt'
DIR_GLOVE = './embeddings/'

embeddings = {}
file1 = open(os.path.join(DIR_GLOVE, filename2), encoding='utf-8')
i = 0
for line in file1:
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings[word] = coefs
    except ValueError:
        i += 1
file1.close()



################################################################################
#Creating Vocab and Data

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 100

tokenizer1 = Tokenizer(num_words=MAX_NB_WORDS)               #tokenizing
tokenizer1.fit_on_texts(sentences)
sequences = tokenizer1.texts_to_sequences(sentences)
vocab = tokenizer1.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)


################################################################################
#creating embedding matrix

MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300     #created a 300-dim embedding

word_index = vocab
embeddings_index = embeddings

number_of_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((number_of_words + 1, EMBEDDING_DIM))  #created a 300-dim embedding, we could have changed this, but we agreed on it
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector                          

embedding_mat = embedding_matrix
#the embedding word vectors of the embedding matrix are then fed to the NN model 


################################################################################
#creating the LSTM model

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation
from keras.layers.embeddings import Embedding

pickle.dump([data, y_labels, embedding_mat], open('/content/drive/My Drive/Colab Notebooks/embedding_matrix.pkl', 'wb'))
print ("Data created")

print("Train Test split")
X_train, X_test, y_train, y_test = train_test_split(data, y_labels, test_size=TEST_SPLIT, random_state=42)

TEST_SPLIT = 0.1        #best after trying 0.1, 0.2, and 0.3
VALIDATION_SPLIT = 0.1  #validation split fixed at 10% for validation dataset and 90% for training
epoch = 40                          #specifying the number of epoch to be 40 for the number of epochs of the LSTM model
embedding_matrix = embedding_mat


lstmmodel = Sequential()
n, embedding_dims = embedding_matrix.shape

lstmmodel.add(Embedding(n, embedding_dims, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
lstmmodel.add(LSTM(128, dropout=0.6, recurrent_dropout=0.6))     #specifying a dropout probability at 0.6 and 128 memory units since it resulted in the best performance of the lstmmodel
lstmmodel.add(Dense(7))                                          # we have 7 different outputs that represent the 7 different classes in the ISEAR dataset
lstmmodel.add(Activation('softmax'))                            #Dense is used for classification
                                        #softmax used to have an output as a 7-dim vector 

lstmmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])    #using optimizer = 'adam' to enhance the performance of gradient decent
print(lstmmodel.summary())
    
lstmmodel.fit(X_train, y_train, validation_split=VALIDATION_SPLIT, epochs=epoch, batch_size=128)    #validation_split applied for hyperparameter tuning.
lstmmodel.save_weights('/content/drive/My Drive/Colab Notebooks/text_lstm_weights.h5')              #

scores= lstmmodel.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (lstmmodel.metrics_names[1], scores[1] * 100))



Data created
Train Test split
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 300)          2700000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 903       
_________________________________________________________________
activation_2 (Activation)    (None, 7)                 0         
Total params: 2,920,551
Trainable params: 220,551
Non-trainable params: 2,700,000
_________________________________________________________________
None
Epoch 1/40
48/48 [==============================] - 17s 346ms/step - loss: 1.8214 - accuracy: 0.2840 - val_loss: 1.5781 - val_accuracy: 0.4165
Epoch 2/40
48/48 [==============================] - 16s 34